In [ ]:
# Global Sustainable Energy & Emissions Analysis (2000–2020)
# Notebook: 01_etl_cleaning.ipynb
# Goal: Load -> Clean -> Validate -> Feature engineer -> Export for Power BI (model + optional star schema)

import os
import numpy as np
import pandas as pd

# -----------------------------
# 0) Paths (edit if needed)
# -----------------------------
RAW_PATH = "global-data-on-sustainable-energy (1).csv"
OUT_DIR = "../data/processed"
os.makedirs(OUT_DIR, exist_ok=True)

# -----------------------------
# 1) Load
# -----------------------------
df = pd.read_csv(RAW_PATH)
print("Raw shape:", df.shape)
df.head()

# -----------------------------
# 2) Standardize column names
# -----------------------------
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# If your dataset uses different names, map them here (edit as needed)
# Common variations seen in global energy datasets:
rename_map = {
    "entity": "country",
    "country_name": "country",
    "nation": "country",
    "iso_code": "iso_code",
    "year": "year",
    "co2": "co2_emissions",
    "co2_emissions": "co2_emissions",
    "co2_emissions_tonnes": "co2_emissions",
    "renewables_share_energy": "renewable_share_energy",
    "renewable_share_energy": "renewable_share_energy",
    "renewables_%": "renewable_share_energy",
    "electricity_generation": "electricity_generation",
    "electricity_demand": "electricity_demand",
    "energy_consumption": "energy_consumption",
    "primary_energy_consumption": "energy_consumption",
    "gdp": "gdp",
    "population": "population",
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

print("Columns:", df.columns.tolist())

# -----------------------------
# 3) Required columns check
# -----------------------------
required = ["country", "year"]
missing_required = [c for c in required if c not in df.columns]
if missing_required:
    raise ValueError(f"Missing required columns: {missing_required}")

# -----------------------------
# 4) Type conversions
# -----------------------------
df["country"] = df["country"].astype("string").str.strip()
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")

# Convert numeric columns safely (only if present)
numeric_candidates = [
    "co2_emissions",
    "renewable_share_energy",
    "energy_consumption",
    "electricity_generation",
    "electricity_demand",
    "gdp",
    "population",
]
for col in numeric_candidates:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# -----------------------------
# 5) Filter to target range + basic cleaning
# -----------------------------
before = len(df)

df = df[df["year"].between(2000, 2020, inclusive="both")]
df = df[df["country"].notna() & (df["country"] != "")]

# Optional: remove aggregate regions if your dataset includes them
# (Keep if you want them in the dashboard; otherwise exclude for country-only analysis)
exclude_entities = {
    "World", "Asia", "Europe", "Africa", "North America", "South America", "Oceania",
    "European Union (27)", "European Union", "High-income countries", "Low-income countries",
    "Upper-middle-income countries", "Lower-middle-income countries"
}
df = df[~df["country"].isin(exclude_entities)]

after = len(df)
print("Rows before:", before, "after:", after, "dropped:", before - after)

# -----------------------------
# 6) Sanity checks + missingness report
# -----------------------------
print("Year range:", df["year"].min(), "-", df["year"].max())
print("Countries:", df["country"].nunique())

missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
missing_pct.head(15)

# -----------------------------
# 7) Feature engineering (dashboard-ready)
# -----------------------------
# 7.1 CO2 per capita (if possible)
if "co2_emissions" in df.columns and "population" in df.columns:
    # Assumption: co2_emissions is in tonnes and population is total people
    df["co2_per_capita"] = np.where(
        (df["population"].isna()) | (df["population"] == 0) | (df["co2_emissions"].isna()),
        np.nan,
        df["co2_emissions"] / df["population"]
    )

# 7.2 Renewable share bounds + clean
if "renewable_share_energy" in df.columns:
    # Clip to [0,100] if percent-based
    df["renewable_share_energy"] = df["renewable_share_energy"].clip(lower=0, upper=100)

# 7.3 YoY change for CO2 and renewable share (within each country)
def add_yoy(series: pd.Series) -> pd.Series:
    return series.pct_change() * 100

if "co2_emissions" in df.columns:
    df = df.sort_values(["country", "year"])
    df["co2_yoy_pct"] = df.groupby("country")["co2_emissions"].apply(add_yoy)

if "renewable_share_energy" in df.columns:
    df = df.sort_values(["country", "year"])
    df["renewable_share_yoy_pct"] = df.groupby("country")["renewable_share_energy"].apply(add_yoy)

# 7.4 Simple category for renewable adoption (optional, useful for slicers)
if "renewable_share_energy" in df.columns:
    df["renewable_adoption_band"] = pd.cut(
        df["renewable_share_energy"],
        bins=[-0.1, 10, 25, 50, 75, 100.1],
        labels=["0–10%", "10–25%", "25–50%", "50–75%", "75–100%"]
    )

# -----------------------------
# 8) Create a clean model export for Power BI (single table)
# -----------------------------
# Keep only columns that exist + are useful
model_cols = [
    "country", "iso_code", "year",
    "co2_emissions", "co2_per_capita", "co2_yoy_pct",
    "renewable_share_energy", "renewable_share_yoy_pct", "renewable_adoption_band",
    "energy_consumption", "electricity_generation", "electricity_demand",
    "gdp", "population"
]
model_cols = [c for c in model_cols if c in df.columns]

df_model = df[model_cols].copy()

out_model = os.path.join(OUT_DIR, "energy_emissions_model_2000_2020.csv")
df_model.to_csv(out_model, index=False)
print("Saved:", out_model, "shape:", df_model.shape)

# -----------------------------
# 9) Optional: Star schema exports for Power BI (recommended for larger models)
# -----------------------------
# DimYear
dim_year = pd.DataFrame({"year": sorted(df_model["year"].dropna().unique())})
dim_year["year_key"] = dim_year["year"].astype(int)

# DimCountry
dim_country_cols = ["country"]
if "iso_code" in df_model.columns:
    dim_country_cols.append("iso_code")

dim_country = df_model[dim_country_cols].drop_duplicates().reset_index(drop=True)
dim_country["country_key"] = dim_country.index + 1

# Fact table
fact = df_model.merge(dim_country, on=dim_country_cols, how="left")
fact["year_key"] = fact["year"].astype(int)

# Choose fact columns
fact_cols = [
    "country_key", "year_key",
    "co2_emissions", "co2_per_capita", "co2_yoy_pct",
    "renewable_share_energy", "renewable_share_yoy_pct",
    "energy_consumption", "electricity_generation", "electricity_demand",
    "gdp", "population"
]
fact_cols = [c for c in fact_cols if c in fact.columns]
fact_energy = fact[fact_cols].copy()

# Save star schema
dim_year.to_csv(os.path.join(OUT_DIR, "dim_year.csv"), index=False)
dim_country.to_csv(os.path.join(OUT_DIR, "dim_country.csv"), index=False)
fact_energy.to_csv(os.path.join(OUT_DIR, "fact_energy.csv"), index=False)

print("Saved star schema files:",
      os.path.join(OUT_DIR, "dim_year.csv"),
      os.path.join(OUT_DIR, "dim_country.csv"),
      os.path.join(OUT_DIR, "fact_energy.csv"))

# -----------------------------
# 10) Quick QA summary for README / insights
# -----------------------------
summary = {
    "countries": int(df_model["country"].nunique()),
    "years": f"{int(df_model['year'].min())}-{int(df_model['year'].max())}",
    "rows": int(len(df_model)),
}
print("Model summary:", summary)

if "co2_emissions" in df_model.columns:
    print("Top 10 CO2 emitters (latest year):")
    latest_year = int(df_model["year"].max())
    top_emitters = (
        df_model[df_model["year"] == latest_year]
        .dropna(subset=["co2_emissions"])
        .groupby("country", as_index=False)["co2_emissions"].sum()
        .sort_values("co2_emissions", ascending=False)
        .head(10)
    )
    display(top_emitters)

if "renewable_share_energy" in df_model.columns:
    print("Top 10 renewable share countries (latest year):")
    latest_year = int(df_model["year"].max())
    top_renewables = (
        df_model[df_model["year"] == latest_year]
        .dropna(subset=["renewable_share_energy"])
        .sort_values("renewable_share_energy", ascending=False)
        .loc[:, ["country", "renewable_share_energy"]]
        .head(10)
    )
    display(top_renewables)
